In [1]:
import librosa
from librosa import display
import matplotlib
import matplotlib.pyplot as plt
import scipy
import os
from mido import MidiFile
import pandas as pd
import numpy as np
import csv
import multiprocessing as mp
import shutil
import sounddevice as sd
import guitarpro as gp
import madmom
import math
import Transcriber
import scipy.io.wavfile
import midi2audio
import midi
import gc


In [2]:
def __editSignalLength(signal,desired_length):
    
    if(len(signal) < desired_length):
        pad_amount = desired_length - len(signal)
        newSignal = np.pad(array=signal,pad_width= (0, pad_amount),mode='constant')
    else:
        newSignal = signal[:desired_length]
        
    return newSignal

def __generateOnsetFrames(pathGPsong,sampling_rate):
    beatList,songLength = Transcriber.transcribeGP3toTab(pathGPsong)
    onsetFrameValues = []
    onsetTimeValues = []
    onsetFrameValues.append(0)    ## avoiding substracting 0, resulting in negative number
    for index,row in enumerate(beatList[1:]):
        onsetTimeValues.append(row[0])
        onsetFrameValues.append(int((onsetTimeValues[index] * sampling_rate) - 220))
    return onsetFrameValues



def __secondToMinute(second):
    minute = 0  
    while(second - 60 >= 0):
        minute += 1
        second -= 60

    return str(f'{minute}:{round(second)}') 


def __checkValidityGeneratedSong(pathGP3,pathWav):   
    beatList,songLength = Transcriber.transcribeGP3toTab(pathGP3)
    sample_rate,data = scipy.io.wavfile.read(pathWav)
    #data,sample_rate = librosa.load(pathGP3,sr=None)
    if(len(beatList[-1][1]) is 0): ## If last beat is a rest
        beatListReversed = beatList[::-1]
        beatListOnlyBeats = [row for row in beatListReversed if(len(row[1]) is not 0)]
        indexLastBeat = beatList.index(beatListOnlyBeats[0])
        generatedLengthSong = beatList[indexLastBeat + 1][0]
    else:
        generatedLengthSong = songLength
    
    if((((len(data) / sample_rate) - generatedLengthSong) <= 0.100) and (generatedLengthSong - (len(data) / sample_rate) <= 0.100)):
        isValid = True
    else:
        isValid = False
    return isValid
    
def generateOnsetSpectograms2(pathAudio,durationTimeSecond,pathSave,GP3Path):
    beatList,songLength = Transcriber.transcribeGP3toTab(GP3Path) #############
    samples,sampling_rate = librosa.load(pathAudio,sr=None,res_type='kaiser_fast')
    onsetFrames = __generateOnsetFrames(GP3Path,sampling_rate)
    durationFrame = int(sampling_rate* durationTimeSecond)

    for index,onset in enumerate(onsetFrames[:-1]):
        onset_padded = __editSignalLength(signal=samples[onset:onsetFrames[index + 1] - 320],desired_length=durationFrame) 
        plt.box(0)
        spectogram = librosa.stft(onset_padded,n_fft=2048,hop_length=512)
        mel_spectogram = librosa.feature.melspectrogram(sr=sampling_rate,S=spectogram,n_fft=2048,hop_length=512,n_mels=128)
        mel_dB = librosa.amplitude_to_db(np.abs(mel_spectogram)) 
        librosa.display.specshow(mel_dB,cmap='gray')
        fig = plt.figure() 
        fig.add_subplot(1,1,1)
        plt.savefig(f'{pathSave}\\{index + 1}.png')
        plt.clf() #################################
        plt.close()
    
    
    onset_padded = __editSignalLength(signal=samples[onsetFrames[-1]:(int(songLength * sampling_rate) - 320)],desired_length=durationFrame) 
    plt.box(0)
    spectogram = librosa.stft(onset_padded,n_fft=2048,hop_length=512)
    mel_spectogram = librosa.feature.melspectrogram(sr=sampling_rate,S=spectogram,n_fft=2048,hop_length=512,n_mels=128)
    mel_dB = librosa.amplitude_to_db(np.abs(mel_spectogram)) 
    librosa.display.specshow(mel_dB,cmap='gray')
    plt.savefig(f'{pathSave}\\{len(onsetFrames)}.png')
    plt.clf() #####################################
    plt.close()


* The problem seems to be related to using pyplot interfaces in matplotlib; perhaps using object-oriented interface would fix this

In [3]:
for GP3File in os.scandir(r'D:\PolyPhonic\data\gp3'):
    pathWav = f"D:\\SharedFolder\\wav\\{GP3File.name.rsplit('.',1)[0]}.wav"
    pathSaveDir = f"D:\\PolyPhonic\\data\\training\\{GP3File.name.rsplit('.gp',1)[0]}"

    if(os.path.isdir(pathSaveDir)):
        print(pathSaveDir," Already Exists")
        
    elif(__checkValidityGeneratedSong(GP3File.path,pathWav)):
        print(GP3File)
        os.mkdir(pathSaveDir)
        generateOnsetSpectograms2(pathAudio=pathWav,durationTimeSecond= 0.500,pathSave=pathSaveDir,GP3Path=GP3File.path)
        gc.collect() ## freeing resources after generating file
    else:
        print(f"{GP3File.name} length not valid")

D:\PolyPhonic\data\training\1 God - Grace.gp4  Already Exists
<DirEntry '1 God - Loom.gp4.gp3'>
<DirEntry '1 God - Sativa Arabica.gp4.gp3'>


KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

# CSV FILE 


In [6]:
path_directory_training= r'D:\PolyPhonic\data\training\images'
path_labels_training= r'D:\PolyPhonic\data\training\labels.csv'

path_labels_valid =  r'D:\PolyPhonic\data\valid\images'
path_directory_valid =   r'D:\PolyPhonic\data\valid\labels.csv'

path_directory_test = r'D:\PolyPhonic\data\test\images'
path_labels_test =     r'D:\PolyPhonic\data\tets\labels.csv'

In [7]:
dictionary_stringFret = {}     ## dictionary for one-hot-encoded values
counter = 0
for string in range(1,7):
    for fret in range(0,25):
        dictionary_stringFret[counter] = (string,fret)
        counter += 1 
dictionary_stringFret_reversed = {v: k for k, v in dictionary_stringFret.items()}
dictionary_stringFret_reversed

{(1, 0): 0,
 (1, 1): 1,
 (1, 2): 2,
 (1, 3): 3,
 (1, 4): 4,
 (1, 5): 5,
 (1, 6): 6,
 (1, 7): 7,
 (1, 8): 8,
 (1, 9): 9,
 (1, 10): 10,
 (1, 11): 11,
 (1, 12): 12,
 (1, 13): 13,
 (1, 14): 14,
 (1, 15): 15,
 (1, 16): 16,
 (1, 17): 17,
 (1, 18): 18,
 (1, 19): 19,
 (1, 20): 20,
 (1, 21): 21,
 (1, 22): 22,
 (1, 23): 23,
 (1, 24): 24,
 (2, 0): 25,
 (2, 1): 26,
 (2, 2): 27,
 (2, 3): 28,
 (2, 4): 29,
 (2, 5): 30,
 (2, 6): 31,
 (2, 7): 32,
 (2, 8): 33,
 (2, 9): 34,
 (2, 10): 35,
 (2, 11): 36,
 (2, 12): 37,
 (2, 13): 38,
 (2, 14): 39,
 (2, 15): 40,
 (2, 16): 41,
 (2, 17): 42,
 (2, 18): 43,
 (2, 19): 44,
 (2, 20): 45,
 (2, 21): 46,
 (2, 22): 47,
 (2, 23): 48,
 (2, 24): 49,
 (3, 0): 50,
 (3, 1): 51,
 (3, 2): 52,
 (3, 3): 53,
 (3, 4): 54,
 (3, 5): 55,
 (3, 6): 56,
 (3, 7): 57,
 (3, 8): 58,
 (3, 9): 59,
 (3, 10): 60,
 (3, 11): 61,
 (3, 12): 62,
 (3, 13): 63,
 (3, 14): 64,
 (3, 15): 65,
 (3, 16): 66,
 (3, 17): 67,
 (3, 18): 68,
 (3, 19): 69,
 (3, 20): 70,
 (3, 21): 71,
 (3, 22): 72,
 (3, 23): 73,
 (3,

In [8]:
with open(r'D:\PolyPhonic\data\test\labels.csv', 'w', newline='') as csvfile:  ##Generating label for every beat of song in a CSVfile
                                                                                   ## named labels
    rowWriter = csv.writer(csvfile, delimiter=',')
    headerRow = ['filename'] + [i for i in range(0,150)]
    rowWriter.writerow(headerRow) 
    for file in os.scandir(r'D:\PolyPhonic\data\test\images'):
        listBeat,time = Transcriber.transcribeGP3toTab(f'D:\\PolyPhonic\\data\\gp3\\{file.name}.gp3')
        for beatIndex,beatRow in enumerate(listBeat):
            row = [f'{file.path}\\{beatIndex + 1}.png']
            one_hot_encoded_notes = np.zeros(150)
            for note in beatRow[1]:
                one_hot_encoded_notes[dictionary_stringFret_reversed[note]] = 1 

            row += one_hot_encoded_notes.tolist()
            rowWriter.writerow(row)        


In [24]:
with open(r'C:\Users\Gebruiker\Desktop\testFile\labels.csv', 'w', newline='') as csvfile:  ##Generating label for every beat of song in a CSVfile
                                                                                   ## named labels
    rowWriter = csv.writer(csvfile, delimiter=',')
    headerRow = ['filename'] + [i for i in range(0,150)]
    rowWriter.writerow(headerRow) 
    
    listBeat,time = Transcriber.transcribeGP3toTab(r'D:\PolyPhonic\data\gp3\Tarrega, Francisco - Capricho arabe (2).gp3')
    for beatIndex,beatRow in enumerate(listBeat):
        row = [f'C:\\Users\\Gebruiker\\Desktop\\testFile\\images\\{beatIndex + 1}.png']
        one_hot_encoded_notes = np.zeros(150)
        for note in beatRow[1]:
            one_hot_encoded_notes[dictionary_stringFret_reversed[note]] = 1 
            
        row += one_hot_encoded_notes.tolist()
        rowWriter.writerow(row)   


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\PolyPhonic\\data\\gp3\\Tarrega, Francisco - Capricho arabe (2).gp3'

In [ ]:
## cutting file extension to 1 type
for gpFile in os.scandir(r'D:\PolyPhonic\data\gp3'):
    try:
        os.rename(src=gpFile.path,dst= f"{gpFile.path.rsplit('.gp',6)[0]}.gp3")
    except FileExistsError:
        print(gpFile.name)

In [23]:
beatList,time = Transcriber.transcribeGP3toTab(r'D:\PolyPhonic\data\gp3\Bach, Johann Sebastian - Minuetto (part 2).gp3')
dictionary_stringFret_reversed[(beatList[0][1][0])]

53

In [11]:
listBeat,time = Transcriber.transcribeGP3toTab(r'D:\PolyPhonic\data\gp3\Tarrega, Francisco - Capricho arabe (2).gp3')
listBeat

[[0, [(4, 7), (5, 7), (6, 7)]],
 [3.4615384615384612, [(1, 15)]],
 [3.614599686028257, [(1, 12)]],
 [3.7676609105180527, [(1, 10)]],
 [3.9207221350078485, [(1, 9)]],
 [4.073783359497645, [(1, 12)]],
 [4.2268445839874405, [(1, 10)]],
 [4.379905808477236, [(1, 6)]],
 [4.532967032967032, [(1, 5)]],
 [4.686028257456828, [(1, 8)]],
 [4.8390894819466235, [(1, 6)]],
 [4.992150706436419, [(2, 8)]],
 [5.145211930926215, [(1, 0)]],
 [5.298273155416011, [(1, 0)]],
 [7.605965463108318, [(2, 3), (3, 3), (4, 5)]],
 [8.471350078492934, [(2, 5)]],
 [8.759811616954472, [(2, 2), (3, 2), (4, 2), (5, 0)]],
 [12.221350078492934, [(4, 7), (5, 7), (6, 7)]],
 [15.682888540031396, [(1, 15)]],
 [15.835949764521192, [(1, 12)]],
 [15.989010989010989, [(1, 10)]],
 [16.142072213500786, [(1, 9)]],
 [16.295133437990582, [(1, 12)]],
 [16.44819466248038, [(1, 10)]],
 [16.601255886970176, [(1, 6)]],
 [16.754317111459972, [(1, 5)]],
 [16.90737833594977, [(1, 8)]],
 [17.060439560439566, [(1, 6)]],
 [17.213500784929362, [(

In [3]:
def findMissingGp3():
    
    for fileGP3 in os.scandir(r"D:\PolyPhonic\data\gp3 double extension"):
        songName = fileGP3.name.split('.',1)[0]
        
        for fileWav in os.scandir(r"D:\PolyPhonic\data\wav"):
            if(songName in fileWav.name is False):
                print(fileWav.name)
                
findMissingGp3()    

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\PolyPhonic\\data\\gp3 double extension'

In [ ]:
def lengthSong(GP3File = None,WavFile = None):
    if(GP3File is not None):
        return Transcriber.transcribeGP3toTab(GP3File)
    if(WavFile is not None):
        sample_rate,data = scipy.io.wavfile.read(WavFile)
        return (len(data) / sample_rate)
# lengthSong(r"D:\PolyPhonic\data\gp3\1 God - Grace.gp4.gp3")
lengthSong(WavFile=r"D:\SharedFolder\wav\1 God - Grace.gp4.wav")
# len(Transcriber.transcribeGP3toTab(r"D:\PolyPhonic\data\gp3\1 God - Grace.gp4.gp3")[0])

In [5]:
beatList, songLength = Transcriber.transcribeGP3toTab(r"D:\PolyPhonic\data\gp3\ACDC - Stiff Upper Lip (Solo).gp3.gp3")
beatList[-1]

[44.15999999999998, []]

In [25]:
def __checkValidityGeneratedSong(pathGP3,pathWav):   
    beatList,songLength = Transcriber.transcribeGP3toTab(pathGP3)
    sample_rate,data = scipy.io.wavfile.read(pathWav)
    #data,sample_rate = librosa.load(pathGP3,sr=None)
    if(len(beatList[-1][1]) is 0):  
        beatListReversed = beatList[::-1]
        beatListOnlyBeats = [row for row in beatListReversed if(len(row[1]) is not 0)]
        indexLastBeat = beatList.index(beatListOnlyBeats[0])
        generatedLengthSong = beatList[indexLastBeat + 1][0]
    else:
        generatedLengthSong = songLength
    
    return (((len(data) / sample_rate)) - generatedLengthSong)

for song in os.scandir(r"D:\PolyPhonic\data\gp3"):
    lengthDifference =  __checkValidityGeneratedSong(song.path,f"D:\\SharedFolder\\wav\\{song.name.rsplit('.gp3',1)[0]}.wav")
    print(lengthDifference)

0.0038095238095365858
0.0045770102869937546
0.0028117913844027953
2.4681986767326975
2.4347710976306587
208.00344671201813
0.004076297185555688
-1.612558869701175
0.004172335600600263
0.0032653061224436897
0.0033826864068942086
0.003628117913621054
0.004806321153978388
0.003659666762843017
0.0027210884347539377
0.004357685103528297
0.00426303854874277
0.00290249433101053
0.00290249433101053
0.00290249433101053
0.00290249433101053
0.00290249433101053
0.0036281179138768493
0.0032653061221736834
0.002902494332687411
0.0036281179130810415
0.004583411345095101
0.003628117913820006


KeyboardInterrupt: 

In [7]:
for song in os.scandir(r"D:\PolyPhonic\data\gp3"):
    print(song.name.rsplit('.gp3',1)[0])

1 God - Grace.gp4
1 God - Loom.gp4
1 God - Sativa Arabica.gp4
1 God - Tali.gp4
1 God - Wierdo.gp4
2pac - Do For Love.gp3
3 Doors Down - Be Like That.gp3
3 Doors Down - Going Down In Flames.gp3
3 Doors Down - Here Without You.gp3
3 Doors Down - This Time.gp4
311 - All Mixed Up.gp3
311 - Light Years.gp3
311 - Loco.gp3
311 - Lucky.gp3
311 - Salsa.gp3
36 CrazyFists - Left Hand Charity.gp4
704 exercises for guitar, 1-96 - 181-190.gp3
704 exercises for guitar, 1-96 - Exercises 1-96.gp3
704 exercises for guitar, 1-96 - Exercises 141-180.gp3
704 exercises for guitar, 1-96 - Exercises 191-334.gp3
704 exercises for guitar, 1-96 - Exercises 335-382.gp3
704 exercises for guitar, 1-96 - Exercises 383-416.gp3
704 exercises for guitar, 1-96 - Exercises 97-140.gp3
8 Foot Sativa - Believer.gp4
8 Foot Sativa - Fuel Set.gp4
8 Foot Sativa - Hate Made Me.gp4
@lex - Arpége en accoustic.gp3
A Perfect Circle - A Stranger.gp3
AA Le Composer - The Life Continue.gp4
Abba - Money,Money,Money.gp4
Abdelaziz Eleroui

Carcassi, Mateo - Little Waltz in C.gp3
Carcassi, Mateo - Opus 60 # 23.gp3
Carcassi, Mateo - Prelude (2).gp4
Carcassi, Mateo - Prélude.gp3
Carcassi, Mateo - Study nr.1.gp3
Carcassi, Mateo - Valse En Do Majeur.gp3
Carcassi, Mateo - Valse Op4 No2.gp3
Carcassi, Mateo - Valse Op4 No3.gp3
Carcassi, Mateo - Valse Op4 No4.gp3
Carcassi, Mateo - Valse Op4 No5.gp3
Carcassi, Mateo - Valse Op4 No6.gp3
Cardoso, Jorge - Choro.gp4
Cardoso, Jorge - Milonga in d minor.gp3
Cardoso, Jorge - Milonga.gp4
Cardoso, Jorge - Tema Negro.gp4
Carey, Mariah - Hero (2).gp3
Carey, Mariah - Hero.gp3
Carlton, Larry - Her Favorite Song.gp4
Carmichael, Hoagy - Lazy River.gp3
Carnes, Kim - Bette Davis Eyes.gp3
Carrara, Emile - Mon Amant de Saint Jean.gp4
Carulli, Ferdinando - Allegretto nº 15.gp3
Carulli, Ferdinando - Allegretto.gp3
Carulli, Ferdinando - Andante.gp3
Carulli, Ferdinando - Andantino #4.gp3
Carulli, Ferdinando - Andantino (2).gp3
Carulli, Ferdinando - Andantino (Siciliana).gp3
Carulli, Ferdinando - Andantin

Graham, Davey - Anji (2).gp3
Granados, Enrique - Dedicatoria.gp4
Granados, Enrique - La Maja De Goya (Tonadilla Number 7).gp4
Granados, Enrique - Spanish Dance No 2. (Oriental).gp4
Granados, Enrique - Spanish Dance No 5 (Andalusa).gp4
Granados, Manuel - Estudio Double Arpegio.gp4
Granados, Manuel - Estudio Tremolo.gp3
Granados, Manuel - Solea De Concierto.gp3
Granados, Manuel - Solea De Concierto.gp4
Granados, Manuel - Solea por Bulerias.gp4
Grand, Philippe - Swing Philnadi's Blues.gp3
Grave Digger - Culledon Muir.gp4
Grave Digger - House Of Horror.gp4
Grave Digger - Killing Time.gp4
Grave Digger - Riffs.gp3
Grave Digger - The Ballad Of Mary.gp4
Grave Digger - Witchhunter.gp3
Graveland - The Gates To The Kingdom Of Darkness (2).gp4
Great White - Mother _'s eyes.gp3
Green Day - 2000 Light Years Away.gp4
Green Day - All by myself (2).gp3
Green Day - Good Riddance (Time of Your Life).gp3
Green Day - Good Riddance.gp3
Green Day - I Was There.gp3
Green Jelly - Three Little Pigs.gp3
Green, T

Lennon, John - My Mummy's Dead.gp4
Lennon, John - Stand By Me (2).gp3
Lennon, Julian - Salt Water.gp4
Lenorman, Gerard - La Ballade Des Gens Heureux.gp3
Leonhard von CALL - Adagio de L Von Call.gp3
Lerich - Prelude.gp3
Less Than Jake - Gainsville Rock City.gp4
Less Than Jake - The Science Of Selling Yourself Short.gp4
Level 42 - Love Games.gp4
Levent, Haluk - Ela Gozlu.gp4
Liebert, ottmar - Barcelona Nights.gp3
Life Of Agony - River Runs Red.gp4
Ligabue - A Che Ora E La Fine Eel Mondo_.gp4
Ligabue - Certe Notti.gp4
Limp Bizkit - Crack Addict.gp4
Limp Bizkit - Everything.gp3
Limp Bizkit - Livin' It Up.gp3
Limp Bizkit - My Way.gp3
Limp Bizkit - Sour.gp3
Lindenberg, Udo - Leider Nur Ein Vakuum.gp4
Link Wray - Rumble.gp4
Linkin Park - Lying From You.gp3
Linkin Park - Ntr_Mission.gp4
Linkin Park - Points Of Authority (Main Riff).gp3
Linnemann, Maria - Little cottage on the island.gp3
Liquid Tension Experiment - Another Dimension.gp4
Liquid Tension Experiment - Hourglass (2).gp3
Liquid Tensi

Powell, Baden - Samba em Preludio.gp3
Powell, Baden - Samba Triste.gp3
Powell, Baden - Samba Triste.gp4
Powell, Baden - Saudades Marcia.gp3
Powell, Baden - Se Todos Fossem Iguais A Voce.gp3
Powell, Baden - Tema Triste.gp3
Powell, Baden - Valsa em Nome.gp3
Powell, Baden - Valsa Sem Nome.gp4
Powerman 5000 - Bombshell (Main Riff).gp3
Pozone, Lucio - Buleria Para Mi Amor.gp4
Presley, Elvis - Can_'t Help Falling in Love (2).gp3
Presley, Elvis - King Creole.gp3
Presley, Elvis - Love Me Tender (3).gp4
Presley, Elvis - Love Me Tender.gp3
Presley, Elvis - Mean Woman Blues.gp3
Pride & Glory - Horse Called War.gp4
Pride & Glory - Lovin' Woman.gp4
Primus - Antipop.gp3
Primus - Buckethead.gp3
Primus - De Anza Jig (2).gp3
Primus - Fish On (bass intro).gp3
Primus - Jerry Was a Race Car Driver (Bass intro).gp3
Primus - Jerry Was a Race Car Driver.gp3
Primus - Laquerhead.gp3
Primus - My Name Is Mud.gp3
Primus - Professer Nutbutter's House Of Treats.gp4
Primus - Seas Of Cheese.gp3
Primus - Those Damned 

Sylent Noyse - The Story.gp4
Symphony X - Dressed To Kill.gp3
Symphony X - Inferno (unleash the Fire) (intro).gp3
Symphony X - The Bird-Serpent War Cataclysm.gp3
System Of A Down - Aerials (3).gp3
System Of A Down - Atwa (2).gp3
System Of A Down - Bounce (2).gp3
System Of A Down - Darts (2).gp3
System Of A Down - Darts.gp3
System Of A Down - Ego Brain (Intro).gp3
System Of A Down - Forest (2).gp3
System Of A Down - Forest (Normal Tuning).gp3
System Of A Down - Forest.gp3
System Of A Down - Innervision (3).gp3
System Of A Down - Innervision.gp3
System Of A Down - Know.gp3
System Of A Down - Lost In Hollwood.gp4
System Of A Down - Prison Song (2).gp3
System Of A Down - Prison Song (outro live).gp3
System Of A Down - Psycho Outro (live at BDO).gp3
System Of A Down - Roulette (11).gp4
System Of A Down - Science.gp3
System Of A Down - Shimmy (2).gp3
System Of A Down - Shimmy (3).gp3
System Of A Down - Shimmy.gp3
System Of A Down - Sorry (Live @ Reading Festival 2003).gp3
System Of A Down - 

In [ ]:
def generateOnsetSpectograms3(pathAudio,durationTimeSecond,pathSave,GP3Path):
    beatList,songLength = Transcriber.transcribeGP3toTab(GP3Path) #############
    samples,sampling_rate = librosa.load(pathAudio,sr=None,res_type='kaiser_fast')
    onsetFrames = __generateOnsetFrames(GP3Path,sampling_rate)
    durationFrame = int(sampling_rate* durationTimeSecond)

    for index,onset in enumerate(onsetFrames[:-1]):
        onset_padded = __editSignalLength(signal=samples[onset:onsetFrames[index + 1] - 320],desired_length=durationFrame) 
        plt.box(0)
        spectogram = librosa.stft(onset_padded,n_fft=2048,hop_length=512)
        mel_spectogram = librosa.feature.melspectrogram(sr=sampling_rate,S=spectogram,n_fft=2048,hop_length=512,n_mels=128)
        mel_dB = librosa.amplitude_to_db(np.abs(mel_spectogram)) 
        librosa.display.specshow(mel_dB,cmap='gray')
        fig = plt.figure() 
        ax = fig.add_subplot(1,1,1) 
        plt.savefig(f'{pathSave}\\{index + 1}.png') 
        plt.clf() #################################
        plt.close()
    
    
    onset_padded = __editSignalLength(signal=samples[onsetFrames[-1]:(int(songLength * sampling_rate) - 320)],desired_length=durationFrame) 
    plt.box(0)
    spectogram = librosa.stft(onset_padded,n_fft=2048,hop_length=512)
    mel_spectogram = librosa.feature.melspectrogram(sr=sampling_rate,S=spectogram,n_fft=2048,hop_length=512,n_mels=128)
    mel_dB = librosa.amplitude_to_db(np.abs(mel_spectogram)) 
    librosa.display.specshow(mel_dB,cmap='gray')
    plt.savefig(f'{pathSave}\\{len(onsetFrames)}.png')
    plt.clf() #####################################
    plt.close()
    
    
    